In [ ]:
from pathlib import Path
import csv
import uuid

# -----------------------------
# Settings (تنظیمات)
# -----------------------------
STUDENT_ID = "YSQD"     # studentID (شناسه شما)
PHONE_ID   = "S21FE"    # phoneID (شناسه گوشی)

# 4-digit numbering (شماره‌گذاری ۴ رقمی)
START_INDEX = 0   # شروع از 0000
PAD = 4           # 0000, 0001, ...

# Only rename existing files; do NOT download anything (فقط تغییر نام)
# Force extension to .jpg by renaming only (تغییر پسوند فقط با rename)
FORCE_JPG_EXTENSION = True

# Class folder names (اسم پوشه‌های کلاس‌ها)
CLASSES = {
    "binder",
    "coffee-mug",
    "computer-keyboard",
    "mouse",
    "notebook",
    "remote-control",
    "soup-bowl",
    "teapot",
    "toilet-tissue",
    "wooden-spoon",
}

# We treat these as "image files" to rename (پسوندهای رایج عکس)
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".webp", ".bmp", ".tif", ".tiff", ".gif"}


def ask_base_path() -> Path:
    raw = input("Enter base folder path (root that contains class folders):\n> ")
    raw = raw.strip().strip('"').strip("'")
    return Path(raw)


def safe_unique(target: Path) -> Path:
    """Avoid overwriting if a filename already exists (جلوگیری از overwrite)."""
    if not target.exists():
        return target
    i = 1
    while True:
        cand = target.with_name(f"{target.stem}_{i}{target.suffix}")
        if not cand.exists():
            return cand
        i += 1


def find_class_folders(root: Path):
    """Find folders whose name is in CLASSES (پیدا کردن پوشه‌های کلاس)."""
    folders = []
    for p in root.rglob("*"):
        if p.is_dir() and p.name in CLASSES:
            folders.append(p)
    # sort for stable processing order (مرتب‌سازی ثابت)
    folders.sort(key=lambda x: str(x).lower())
    return folders


def list_images_in_folder(folder: Path):
    """List image files directly inside folder (لیست عکس‌ها داخل همان پوشه)."""
    files = []
    for p in folder.iterdir():
        if p.is_file() and p.suffix.lower() in IMAGE_EXTS:
            files.append(p)

    # Stable order: by modified time then name (ترتیب ثابت)
    files.sort(key=lambda x: (x.stat().st_mtime, x.name.lower()))
    return files


def rename_images_in_class_folder(folder: Path, log_rows: list):
    """
    Rename all existing images in ONE class folder to:
    STUDENT_ID_PHONE_ID_classname_0000.jpg, 0001.jpg, ...
    (فقط همان فایل‌های موجود rename می‌شوند)
    """
    class_name = folder.name
    images = list_images_in_folder(folder)
    if not images:
        return

    print(f"\n=== {class_name} ===")
    print(f"Folder: {folder}")
    print(f"Found {len(images)} images -> renaming with 4-digit indices")

    # Step A: rename everything to temporary names to avoid collisions
    tmp_map = []  # (tmp_path, original_name)
    for p in images:
        tmp = folder / f"__tmp__{uuid.uuid4().hex}{p.suffix.lower()}"
        p.rename(tmp)
        tmp_map.append((tmp, p.name))

    # Step B: rename temp files to final standardized names
    idx = START_INDEX
    for tmp_path, old_name in tmp_map:
        num = str(idx).zfill(PAD)  # 4 digits
        ext = ".jpg" if FORCE_JPG_EXTENSION else tmp_path.suffix.lower()
        new_name = f"{STUDENT_ID}_{PHONE_ID}_{class_name}_{num}{ext}"
        new_path = safe_unique(folder / new_name)

        tmp_path.rename(new_path)

        log_rows.append({
            "folder": str(folder),
            "old_name": old_name,
            "new_name": new_path.name
        })
        idx += 1


def main():
    base = ask_base_path()
    if not base.exists():
        print("ERROR: base path does not exist:", base)
        return

    class_folders = find_class_folders(base)
    if not class_folders:
        print("No class folders found under:", base)
        print("Expected folder names like:", sorted(CLASSES))
        return

    log_rows = []
    print(f"\nFound {len(class_folders)} class folders. Renaming ONLY existing files...")

    for folder in class_folders:
        rename_images_in_class_folder(folder, log_rows)

    # Write log (گزارش تغییر نام)
    log_path = base / "rename_log.csv"
    with open(log_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["folder", "old_name", "new_name"])
        writer.writeheader()
        writer.writerows(log_rows)

    print("\nDONE ✅")
    print("Log saved to:", log_path)


if __name__ == "__main__":
    main()
